In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime as dt
import requests
import json

In [85]:
def get_candles_history(ticker: str, tf: str, n_tf: int):
    """
    Get candlestick history of a ticker from OKX 
    """
    candles = []
    min_ts = None
    for b in range((n_tf + 99) // 100):
        params = {
            'instId': ticker,
            'bar': tf,
        }
        if min_ts is not None:
            params['after'] = str(min_ts)
        responce = requests.get('https://www.okx.com/api/v5/market/history-mark-price-candles', params)
        if not responce.ok:
            print('Bad request')
            return
        for cs in json.loads(responce.text)['data']:
            if min_ts is None or min_ts > int(cs[0]):
                min_ts = int(cs[0])
            candle = {
                'dt': dt.datetime.fromtimestamp(int(cs[0]) // 1000),
                'o': float(cs[1]),
                'h': float(cs[2]),
                'l': float(cs[3]),
                'c': float(cs[4]),
            }
            candles.append(candle)
    df_candles = pd.DataFrame(candles[::-1])
    return df_candles.head(n_tf)

In [95]:
tickers = ['BTC-USDT', 'ETH-USDT', 'DOT-USDT', 'OKB-USDT', 'XRP-USDT',
           'SOL-USDT', 'DOGE-TRX', 'TRX-USDT', 'LTC-USDT', 'TON-USDT',
          ]
tf = '1D'
n_tf = 1000
data = []
for ticker in tickers:
    df = get_candles_history(ticker, tf, n_tf)
    if df.shape[0] > 0:
        ts = pd.Series(data=df['c'].values, index=df['dt'], name=ticker)
        data.append(ts)
data = pd.concat(data, axis=1)#.reset_index()
data.head()

,BTC-USDT,ETH-USDT,DOT-USDT,OKB-USDT,XRP-USDT,SOL-USDT,TRX-USDT,LTC-USDT,TON-USDT
dt,,,,,,,,,
2022-06-11 19:00:00,28092.7,1526.35,7.9915,11.815,0.36039,34.014,0.07815,51.32,NaN
2022-06-12 19:00:00,23265.8,1229.81,6.9590,10.830,0.31385,27.909,0.06425,43.18,NaN
2022-06-13 19:00:00,22692.4,1251.33,7.4640,11.230,0.32180,31.051,0.05935,45.60,NaN
2022-06-14 19:00:00,21415.2,1119.99,7.3022,11.062,0.31179,29.507,0.05523,44.97,NaN
2022-06-15 19:00:00,21072.4,1113.98,7.4287,11.036,0.31678,31.323,0.06019,45.59,NaN


In [96]:
data.describe()

,BTC-USDT,ETH-USDT,DOT-USDT,OKB-USDT,XRP-USDT,SOL-USDT,TRX-USDT,LTC-USDT,TON-USDT
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,802.000000
mean,45131.224500,2244.247780,6.121652,40.702138,0.709715,86.676584,0.111074,78.050800,3.700155
std,25670.290985,789.947014,1.500657,13.625767,0.628805,71.862331,0.057901,18.227604,1.987787
min,16133.800000,989.840000,3.632000,10.241000,0.305490,9.384000,0.049320,43.180000,1.194000
25%,23716.400000,1629.580000,4.958250,36.640000,0.434213,22.673500,0.066513,64.902500,2.133000
50%,34025.550000,1941.695000,6.074050,43.320000,0.517330,42.385500,0.093000,73.680000,2.422000
75%,64148.025000,2915.112500,7.047750,50.040000,0.604200,147.470000,0.131735,89.190000,5.462750
max,106477.000000,4019.470000,11.163000,69.070000,3.307400,270.580000,0.351290,143.610000,7.983000


In [ ]:
plt.plot(